In [34]:
import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
%matplotlib inline

In [35]:
original_df = pd.read_excel('/home/quark/Documents/projects/TFM/Databases/datos.xlsx')
original_df.sort_values(['Timestamp'], ascending=True, inplace=True)
original_df['Timestamp'] = pd.to_datetime(original_df['Timestamp'], unit='s')

marag_df = original_df[original_df['SiteId'] == 2][['Timestamp', 'TotalEntries']]
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)
marag_df = marag_df['2017-01-09 00:00:00':]
#Since from this date the time series is constinuous, we slice our dataframe consequently

##### Walk-forward validation

In [112]:
#First we need to split the raw data

def X_data_walk_forward(X_data):
    cut1 = int(0.6*len(X_data))
    X_train_raw = X_data[:cut1]
    return X_train_raw
X_train_raw = X_data_walk_forward(marag_df)

In [127]:
#Transform

#Data transformation
#define method to transform data, for example, create lags:
def create_lags(dataframe, N):
    temp_df = dataframe.copy()
    for i in range(1,N+1):
        temp_df['t-' + str(i)] = temp_df['TotalEntries'].shift(i)
    
    return temp_df.iloc[N:]
df_with_lags = create_lags(marag_df, 48)

In [129]:
df_with_lags.head()

,TotalEntries,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,...,t-39,t-40,t-41,t-42,t-43,t-44,t-45,t-46,t-47,t-48
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-10 00:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 00:30:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 01:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 01:30:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 02:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
###Note: it's recommended to use TimSeries Split, but for the time being we'll use train_test_split

#Initial data preparation:
def train_dev_test_split(X_data, y_data):
    
##CODE IMPROVEMENT: raise error if X and y of different size!    
    cut1 = int(0.6*len(X_data))
    cut2 = ((len(X_data)-cut1)//2) + cut1
    X_train = X_data[:cut1]
    X_dev = X_data[cut1:cut2]
    X_test = X_data[cut2:]
    y_train = y_data[:cut1]
    y_dev = y_data[cut1:cut2]
    y_test = y_data[cut2:]
    
    return X_train, X_dev, X_test, y_train, y_dev, y_test
    
#X_train, X_temp, y_train, y_temp = train_test_split(men_df.drop('TotalEntriesMen', axis = 1), men_df['TotalEntriesMen'], test_size = 0.4, random_state = 42, shuffle=False)
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(df_with_lags.drop(['TotalEntries'], axis=1), df_with_lags['TotalEntries'])


In [131]:
X_train.head()

,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,...,t-39,t-40,t-41,t-42,t-43,t-44,t-45,t-46,t-47,t-48
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-10 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
#Fit model

#Define model to be used
model = LinearRegression().fit(X_train, y_train)

In [111]:
#Walk-forward validation

def walk_forward_validation(X_train, y_train, X_test, L, model):

    '''This function integrates a walk-forward validation method for multiple models.
    Parameters: model, #lags, train data, test data, length of test_data'''

    X_grow = X_train_raw.astype('float64').sort_index()
    y_grow_hat = []
    RMSE_record = []
    for i in range(len(X_test)):
        y_hat = model.predict(X_test[i:i+1])
        RMSE_record.append(y_hat)
        y_grow_hat.append(y_hat)
        y_hat = pd.DataFrame(y_hat, index=X_dev[i:(i+1)].index, columns=['TotalEntries'])
        #y_hat = pd.DataFrame(y_hat, columns=['TotalEntries'])
        #y_hat.set_index(X_dev[i:(i+1)].index, inplace=True)
        X_grow = pd.concat([X_grow, y_hat])
        X_data = create_lags(X_grow, L)
        model.fit(X_data.drop(['TotalEntries'], axis=1), X_data['TotalEntries'])
        #y_hat_trans = create_lags(pd.DataFrame(y_hat), 49)
        #X_grow = pd.concat(X_grow, y_hat_trans)
        #model.fit(X_grow)
    
    



    
    
    
    
    #def recurrent_prediction(y_train, n_steps, reg):
    #x_dev = list(y_train[-1:-(n_steps +1):-1])
    #y_dev_hat = []
    #for i in range(n_steps):
     #   y_hat = reg.predict([x_dev])[0]
     #   y_dev_hat.append(y_hat)
     #   x_dev.pop(-1)
     #   x_dev = [y_hat] + x_dev
    #return y_dev_hat

#y_test_hat = recurrent_prediction(y_train, 48, lr_model)
#np.sqrt(mean_squared_error(y_test[:48], y_test_hat))

In [103]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression().fit(X_train, y_train)
y_hat = lr_model.predict(X_dev[:1])

In [104]:
y_hat

array([-8.52651283e-14])

In [105]:
y_hat = pd.DataFrame(y_hat, index=X_dev[:1].index, columns=['TotalEntries'])
y_hat

,TotalEntries
Timestamp,
2018-04-20 04:30:00,-8.526513e-14


In [58]:
y_hat = pd.DataFrame(y_hat, columns=['TotalEntries'])
y_hat

,TotalEntries
0,-8.526513e-14


In [101]:
y_hat = pd.DataFrame(y_hat, index=X_dev[:1].index)
y_hat

,0
Timestamp,
2018-04-20 04:30:00,-8.526513e-14


In [97]:
y_hat = pd.DataFrame(y_hat).set_index(X_dev[:1].index, inplace=True)
y_hat

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [88]:
type(y_train)

pandas.core.series.Series

In [89]:
y_train[-10:]

Timestamp
2018-04-19 23:30:00    33
2018-04-20 00:00:00     7
2018-04-20 00:30:00     0
2018-04-20 01:00:00     0
2018-04-20 01:30:00     0
2018-04-20 02:00:00     0
2018-04-20 02:30:00     0
2018-04-20 03:00:00     0
2018-04-20 03:30:00     0
2018-04-20 04:00:00     0
Name: TotalEntries, dtype: int64

In [102]:
pd.concat([y_train, (y_hat)])[-10:]

,0
Timestamp,
2018-04-20 00:00:00,7.000000e+00
2018-04-20 00:30:00,0.000000e+00
2018-04-20 01:00:00,0.000000e+00
2018-04-20 01:30:00,0.000000e+00
2018-04-20 02:00:00,0.000000e+00
2018-04-20 02:30:00,0.000000e+00
2018-04-20 03:00:00,0.000000e+00
2018-04-20 03:30:00,0.000000e+00
2018-04-20 04:00:00,0.000000e+00
